In [5]:
import json
import numpy as np
import tiktoken

# model = "o3-mini"
# model = "gpt-4o"
model = "research-grok-3-latest"
# model = 
file_name = f"/data/long_phan/forecasting_platform/backend/src/benchmarks/new_output/{model}_1101_val_min20_full.json"

with open(file_name, "r") as f:
    data = json.load(f)

cors = []
brier_scores = []
total_tokens = 0
total_cost = 0

enc = tiktoken.encoding_for_model("gpt-4o")

human_cors = []
human_brier_scores = []
no_forecasts = []
for q in data:
    label = q["resolution"]
    pred = q["predictions"][0]
    if not q.get('responses'):
        continue
    if pred is None:
        continue
    if pred > 100:
        continue
    if pred > 1:
        # print(pred)
        # continue
        pred = pred / 100

    
    cor = (pred >= 0.5) == label

    crowd_pred = q["crowd_pred"]
    response = q['responses']

    input_tokens = len(enc.encode(str(response)))
    total_tokens += input_tokens

    # print(crowd_pred)
    total_forecasts = crowd_pred[0][2]
    human_pred = crowd_pred[0][-1]
    human_cor = (human_pred > 0.5) == label

    no_forecasts.append(total_forecasts)

    # if human_pred < 0.1 or human_pred > 0.9:
    #     continue


    cors.append(cor)
    brier_scores.append(np.square(pred - label))

    human_cors.append(human_cor)
    human_brier_scores.append(np.square(human_pred - label))

print(np.mean(cors), np.mean(brier_scores), len(cors))
print(np.mean(human_cors), np.mean(human_brier_scores), len(human_cors))
print(np.mean(no_forecasts))


0.7613636363636364 0.1435067424242424 264
0.803030303030303 0.12780210195230354 264
77.11742424242425


In [ ]:
0.7642585551330798 0.14801391349809886 263
0.8022813688212928 0.12756499819358863 263
77.33460076045627

In [3]:
import json
import numpy as np
import tiktoken

# model = "o3-mini"
model = "gpt-4o"
model = "grok-3-beta"
cors = []
brier_scores = []
total_tokens = 0
total_cost = 0

enc = tiktoken.encoding_for_model("gpt-4o")

human_cors = []
human_brier_scores = []
no_forecasts = []
for file_type in ["test", "val"]:
    file_name = f"/data/long_phan/forecasting_platform/backend/src/benchmarks/new_output/{model}_1101_{file_type}_min20_grok_prompt_full.json"

    with open(file_name, "r") as f:
        data = json.load(f)


    for q in data:
        label = q["resolution"]
        pred = q["predictions"][0]
        if not q.get('responses'):
            continue
        if pred is None:
            continue
        if pred > 100:
            continue
        if pred > 1:
            # print(pred)
            # continue
            pred = pred / 100

        
        cor = (pred >= 0.5) == label

        crowd_pred = q["crowd_pred"]
        response = q['responses']

        input_tokens = len(enc.encode(str(response)))
        total_tokens += input_tokens

        # print(crowd_pred)
        total_forecasts = crowd_pred[0][2]
        human_pred = crowd_pred[0][-1]
        human_cor = (human_pred > 0.5) == label

        no_forecasts.append(total_forecasts)

        # if human_pred < 0.1 or human_pred > 0.9:
        #     continue


        cors.append(cor)
        brier_scores.append(np.square(pred - label))

        human_cors.append(human_cor)
        human_brier_scores.append(np.square(human_pred - label))

print(np.mean(cors), np.mean(brier_scores), len(cors))
print(np.mean(human_cors), np.mean(human_brier_scores), len(human_cors))
print(np.mean(no_forecasts))


0.8026565464895635 0.13523046658444024 527
0.8121442125237192 0.13048155256178845 527
76.21062618595825


In [31]:
import json
import numpy as np
import random
from datetime import datetime

# Dictionary to store results by question ID
results_by_question = {}

models = ["gpt-4o", "grok-3-beta"]
file_types = ["test", "val"]

# Process each model's predictions
for model in models:
    for file_type in file_types:
        file_name = f"new_output/{model}_1101_{file_type}_min20_full.json"
        
        try:
            with open(file_name, "r") as f:
                data = json.load(f)
                
            for q in data:
                q_id = q.get("id", q.get("question_id", str(hash(q["question"]))))
                
                if q_id not in results_by_question:
                    # Initialize entry with question metadata
                    results_by_question[q_id] = {
                        "question": q["question"],
                        "resolution": q["resolution"],
                        "crowd_pred": q["crowd_pred"][0][-1] if q["crowd_pred"] and q["crowd_pred"][0] else None,
                        "forecast_timestamp": q["crowd_pred"][0][0] if q["crowd_pred"] else None,
                        "forecast_date": q["crowd_pred"][0][1] if q["crowd_pred"] and len(q["crowd_pred"][0]) > 1 else None,
                        "models": {}
                    }
                
                # Get this model's prediction
                pred = q["predictions"][0]
                
                # Handle invalid predictions
                if pred is None:
                    continue
                if pred > 100:
                    continue
                if pred > 1:
                    pred = pred / 100
                
                # Add this model's prediction
                results_by_question[q_id]["models"][model] = pred
        except FileNotFoundError:
            print(f"File not found: {file_name}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# Sample 10 questions that have predictions from both models
complete_questions = [q_id for q_id, data in results_by_question.items() 
                     if len(data["models"]) == len(models) and data["crowd_pred"] is not None]

if complete_questions:
    sampled_questions = random.sample(complete_questions, min(10, len(complete_questions)))
    
    # Display the sampled questions in the requested format
    print("Forecast Date\tQuestion\tCrowd Prediction\t4o\tGrok-3-beta")
    print("-" * 100)
    
    for q_id in sampled_questions:
        q_data = results_by_question[q_id]
        
        # Format the forecast date if it's a timestamp
        forecast_date = q_data["forecast_date"]
        if forecast_date and isinstance(forecast_date, (int, float)):
            forecast_date = datetime.fromtimestamp(forecast_date).strftime('%Y-%m-%d')
        
        # Format the row
        row = [
            str(forecast_date),
            q_data["question"][:50] + "..." if len(q_data["question"]) > 50 else q_data["question"],
            str(round(q_data["crowd_pred"], 3)) if q_data["crowd_pred"] is not None else "N/A",
            str(round(q_data["models"].get("gpt-4o", 0), 3)),
            str(round(q_data["models"].get("grok-3-beta", 0), 3))
        ]
        
        print("\t".join(row))
else:
    print("No questions with complete data found.")

# # Print some statistics
# print("\nSummary Statistics:")
# print(f"Total questions: {len(results_by_question)}")
# print(f"Questions with complete data: {len(complete_questions)}")

# # Calculate overall accuracy and Brier scores by model
# for model in models:
#     model_cors = []
#     model_brier = []
    
#     for q_id, data in results_by_question.items():
#         if model in data["models"]:
#             pred = data["models"][model]
#             label = data["resolution"]
            
#             cor = (pred >= 0.5) == label
#             model_cors.append(cor)
#             model_brier.append(np.square(pred - label))
    
#     print(f"\n{model} accuracy: {np.mean(model_cors):.4f}, Brier score: {np.mean(model_brier):.4f}")

# # Human performance
# human_cors = []
# human_brier = []

# for q_id, data in results_by_question.items():
#     if data["crowd_pred"] is not None:
#         human_pred = data["crowd_pred"]
#         label = data["resolution"]
        
#         human_cor = (human_pred > 0.5) == label
#         human_cors.append(human_cor)
#         human_brier.append(np.square(human_pred - label))

# print(f"\nHuman accuracy: {np.mean(human_cors):.4f}, Brier score: {np.mean(human_brier):.4f}")

Forecast Date	Question	Crowd Prediction	4o	Grok-3-beta
----------------------------------------------------------------------------------------------------
2024-12-06	Will New Delhi experience a "Hazardous" air qualit...	0.503	0.35	0.3
2024-11-03	Will there be a bilateral cease-fire or peace agre...	0.036	0.15	0.15
2024-11-02	Will there be a white Christmas in at least 4 of t...	0.169	0.55	0.3
2024-12-09	Will Turkey be a NATO member continuously until Ja...	0.964	0.95	0.98
2024-11-05	Will TikTok US be sold to a US entity before 2025?	0.06	0.35	0.2
2024-11-07	Will John Thune succeed Mitch McConnell as Republi...	0.655	0.75	0.65
2024-11-02	Will Ethiopia formally recognise Somaliland in 202...	0.216	0.15	0.2
2024-11-04	Will Oregon voters approve Measure 118, which woul...	0.222	0.45	0.3
2024-11-01	Will it rain more than 100mm in Brasília, Brazil i...	0.83	0.85	0.85
2025-01-14	Will there be any active, large, non-contained fir...	0.256	0.85	0.6


In [33]:
random.seed(42)
if complete_questions:
    # sampled_questions = random.sample(complete_questions, min(50, len(complete_questions)))
    sampled_questions = complete_questions

    sampled_questions = sorted(sampled_questions, key=lambda q_id: results_by_question[q_id]["forecast_timestamp"])


    # Create a tab-separated output that can be directly copied to Google Sheets
    print("Forecast Date\tQuestion\tResolution\tCrowd Prediction\tGPT-4o\tGrok-3-beta")
    
    for q_id in sampled_questions:
        q_data = results_by_question[q_id]
        
        # Format the forecast date if it's a timestamp
        forecast_date = q_data["forecast_date"]
        if forecast_date and isinstance(forecast_date, (int, float)):
            forecast_date = datetime.fromtimestamp(forecast_date).strftime('%Y-%m-%d')
        
        # Format the row with tab separators for easy copying to spreadsheets
        row = [
            str(forecast_date),
            q_data["question"].replace("\n", " "),
            str(int(q_data["resolution"])),
            str(round(q_data["crowd_pred"], 3)) if q_data["crowd_pred"] is not None else "N/A",
            str(round(q_data["models"].get("gpt-4o", 0), 3)),
            str(round(q_data["models"].get("grok-3-beta", 0), 3)),
        ]
        
        print("\t".join(row))
else:
    print("No questions with complete data found.")

Forecast Date	Question	Resolution	Crowd Prediction	GPT-4o	Grok-3-beta
2024-11-01	Will North Dakota voters approve Initiated Measure 5, which would legalize recreational marijuana in the state?	0	0.419	0.45	0.22
2024-11-01	Will Donald Trump outperform Marsha Blackburn in Tennessee in the 2024 election?	1	0.688	0.6	0.85
2024-11-01	Will Kalshi be #1 in the free apps in the Apple App Store on November 5, 2024?	1	0.157	0.05	0.2
2024-11-01	Will Donald Trump sweep all 7 battleground states in the 2024 election?	1	0.118	0.05	0.12
2024-11-01	Will Hawaii voters approve the constitutional amendment on the ballot in 2024, which would repeal the state's ban on same-sex marriage?	1	0.789	0.85	0.85
2024-11-01	Will the US unemployment rate be above 4% in November 2024?	1	0.729	0.45	0.65
2024-11-01	Will South Dakota voters approve Initiated Measure 29, which would legalize recreational marijuana in the state?	0	0.377	0.15	0.38
2024-11-01	Will Democratic Congressman Don Davis be re-elected in North Caro